# Wide Reciever Data Cleaning

In [1]:
# import packages
import os
import pandas as pd
import numpy as np
import re
import sys
sys.path.append('../src/')
from helpers import *

In [2]:
# Import all data
# assign path
path, dirs, files = next(os.walk("../data/raw_data/wr_stats/"))
file_count = len(files)
# create empty list
dataframes_list = []
 
# append datasets to the list
for i in range(file_count):
    temp_df = pd.read_csv("../data/raw_data/wr_stats/"+files[i], thousands=',')
    dataframes_list.append(temp_df)
    
# Array to store dataframe names
dataframes_names = []

for i in range(len(dataframes_list)):
    # If a player value is null, drop the row
    dataframes_list[i] = dataframes_list[i].dropna(subset=['Player'])
   
   # Rename all the dataframes as name of original csv file with date at the end (i.e. 2018_qb_stats.csv --> qb_stats_2018)
    globals()[files[i][5:].removesuffix('.csv')+'_'+files[i][:4]] = dataframes_list[i]
    dataframes_names.append(files[i][5:].removesuffix('.csv')+'_'+files[i][:4])

In [3]:
#Convert Percentage String to Percentage Float
# float(x.strip('%'))/100
def percent (row):
    if row['% TM'] == None:
        return 0
    return float(row['% TM'].strip('%'))/100

def merge(df1, df2):
        # # merge stats and advanced stats
        df3= pd.merge(left=df1, right=df2, on='Player', how='left').fillna(0)
        
        drop_y(df3)
        rename_x(df3)
        
        # Fix Names
        df3.insert(2, 'NAME', df3.apply (lambda row: team_remove(row), axis=1))
        df3 = df3.drop(columns='Player')
        
        # Adjust Target Share
        df3.insert(2, 'TGT_SHARE', df3.apply (lambda row: percent(row), axis=1))
        df3 = df3.drop(columns='% TM')
    
        
        # Convert all objects to floats
        cols = df3.columns
        df3[cols[3:]] = df3[cols[3:]].apply(pd.to_numeric, errors ='coerce')
        
        return df3

In [4]:
for i in range(len(dataframes_names)-1):
    if dataframes_names[i][-4:] == dataframes_names[i+1][-4:]:
        df1 = globals()[dataframes_names[i]] 
        df2 = globals()[dataframes_names[i+1]]
        
        # rename columns in stats dataframe
        df1 = df1.rename(
            columns={'Rank':'RANK', 'YDS':'REC_YDS', 'RZ TGT': 'RZ_TGT'}).drop(
                columns=['10+ YDS', '20+ YDS', '30+ YDS', '40+ YDS', '50+ YDS', 'CATCHABLE', 'DROP', 'LNG', 'YBC', 'YBC/R'])
        df2 = df2.rename(
            columns={'Rank':'RANK', 'YDS':'REC_YDS', 'TD':'REC_TD', 'ATT':'RUS_ATT', 'YDS.1':'RUS_YDS', 'TD.1':'RUS_TD',}).drop(
                columns=['ROST', '20+', 'LG'])
            
        # Return new dataframe
        globals()[dataframes_names[i+1]] = merge(df1, df2)
        # 
# Change dataframes_names to contian only the mutated dataframes
dataframes_names = dataframes_names[1::2]

In [5]:
# export new csv files
for df in dataframes_names:
    globals()[df].to_csv(f'../data/clean_data/wr_stats_clean/{df}_clean.csv', index=False)